**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json


from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)
        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The function ```act``` takes as input the initial state and returns the new state, the reward and wether the game ends or not. An action is taken into the set of action ```n_action```. The algorithm has two possible paths : either $train = True$ and the function can try actions and learn what is the output of choosing this action, or $train = False$ and the algorithm only outputs the actions that have been learned during the training process.
> During the training process, the value of epsilon allows the agent to "try" some actions that are not stored as learned actions. Thanks to that epsilon, the agent can learn the results of choosing another action.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of Jerry
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1 #line changed

        self.position[self.x, self.y] = 1
        if action == 0: #right
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: #left
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2: #up
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: #down
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=61 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> The two arrays have a shape of $(grid\_size + 4)*(grid\_size + 4)$.
>
> The board is necesary to store the positions of the bonus and malus cells. at the beginning. It is updated during the function ```act```.
> The array ```position``` stores the position of the user and unables the cells on the edge of the grid (filled with -1).
> Finally these two arrays are concatenated to give the new state.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        "outputs a random value between 0 and 3"
        a = np.random.randint(0, 4)
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward            

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/20.0. Average score (-10.0)
Win/lose count 8.0/13.0. Average score (-7.5)
Win/lose count 10.0/12.0. Average score (-5.666666666666667)
Win/lose count 12.0/15.0. Average score (-5.0)
Win/lose count 6.0/17.0. Average score (-6.2)
Win/lose count 11.0/9.0. Average score (-4.833333333333333)
Win/lose count 12.5/12.0. Average score (-4.071428571428571)
Win/lose count 9.5/16.0. Average score (-4.375)
Win/lose count 10.0/13.0. Average score (-4.222222222222222)
Win/lose count 11.5/15.0. Average score (-4.15)
Win/lose count 12.0/13.0. Average score (-3.8636363636363638)
Final score: -3.8636363636363638


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




> Starting from the formula of the $Q$_function :
$$Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] $$
Then 
$$ Q^\pi(s,a)=E_{p^{\pi}}[r(s_{0},a_{0}) + \sum_{t=1}^{\infty}{\gamma^{t}r(s_{t},a_{t})}|s_{0}=s,a_{0}=a] $$
Let's introduce $s$', $a$' such as for any $t$>0, $(s'_{t},a'_{t}) = (s_{t+1},a_{t+1})$. This change of variable implies following $p^\pi(.|s, a)$, thus:
$$ Q^\pi(s,a)=E_{p^{\pi}}[r(s,a)] + E_{(s',a')\sim p(.|s,a)}[\sum_{t=0}^{\infty}{\gamma^{t+1}r(s'_{t},a'_{t})}|s'_{0}=s_{1},a'_{0}=a_{1}] $$
As $r(s,a)$ = cst and $E_{(s',a')\sim p(.|s,a)}[\sum_{t=0}^{\infty}{\gamma^{t}r(s'_{t},a'_{t})}|s'_{0}=s_{1},a'_{0}=a_{1}] = Q^{\pi}(s',a')$
$$ Q^\pi(s,a)= E_{(s',a')\sim p(.|s,a)}[r(s,a) +\gamma * \sum_{t=0}^{\infty}{\gamma^{t}r(s'_{t},a'_{t})}|s'_{0}=s_{1},a'_{0}=a_{1}] $$
$$ Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$
$$ $$
$$ $$
We have 
$$ Q^*(s,a)=\max_{\pi}Q^\pi(s,a)$$
As $a'$ is the only variable which depends on $\pi$, we can maximize with respect to $a'$. Thus :
$$ Q^*(s,a)=\max_{a'}E_{(s',a')\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$
$$ Q^*(s,a)=E_{(s',a')\sim \pi^*(.|s,a)}[\max_{a'}r(s,a)+\max_{a'}\gamma Q^{\pi}(s',a')] $$
$$ Q^*(s,a)=E_{(s',)\sim \pi^*(.|s,a)}[r(s,a)+\max_{a'}\gamma Q^{\pi}(s',a')] $$
$$ $$
$$ $$
Then a plausible objective would be : 
$$ \mathcal{L}(\theta) = \Vert Q^*(s,a,\theta)-Q(s,a,\theta)\Vert^{2}$$
$$ \mathcal{L}(\theta) = \Vert E_{(s',)\sim \pi^*(.|s,a)}[r+\max_{a'}\gamma Q^{\pi}(s',a',\theta)]-Q(s,a,\theta)\Vert^{2}$$
$$ \mathcal{L}(\theta) = E_{(s',)\sim \pi^*(.|s,a)} \Vert r+\max_{a'}\gamma Q^{\pi}(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [10]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        "idea : if len(memory)>max then we delete the latest"
        self.memory = self.memory[1-self.max_memory:] + [m]

    def random_access(self):
        index = np.random.randint(0, len(self.memory))
        #choice = self.memory[index]
        #del self.memory[index]
        #self.memory.append(choice)
        return self.memory[index]

***
The pipeline we will use for training is given below:

In [11]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        #Update of the agent.Learning_rate
        if (e%10 == 0):
            agent.Learning_rate = 1/(e/10+1)
        
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        # Learning rate for the reinforce function
        self.Learning_rate = 1 #0.6
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, 0))[0])
        

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        for i in range(self.batch_size):
            ######## FILL IN
            s, n_s, a, r, game_over = self.memory.random_access()
            input_states[i] = s.copy()
            target = self.model.predict(np.expand_dims(s, 0))[0]
            
            if game_over_:
                ######## FILL IN
                target[a] = r
            else:
                ######## FILL IN
                n_target= self.model.predict(np.expand_dims(n_s, 0))[0]
                target[a] = target[a] + self.Learning_rate*(r + self.discount*max(n_target)-target[a])
            target_q[i, :] = target.copy()
            
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,n_state,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        self.lr = lr
        self.n_state=n_state
        # NN Model
        model = Sequential()
        model.add(Reshape((5*5*n_state, ), input_shape = (5, 5, n_state)))
        model.add(Dense(25, activation = "relu"))
        model.add(Dense(25, activation = "relu"))
        model.add(Dense(4, activation = "linear"))
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(Adam(lr=lr), "mse")
        self.model = model

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.001, n_state = 2, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train50.mp4'))

Epoch 000/061 | Loss 1.6318 | Win/lose count 6.5/4.0 (2.5)
Epoch 001/061 | Loss 1.6850 | Win/lose count 2.5/8.0 (-5.5)
Epoch 002/061 | Loss 1.7802 | Win/lose count 3.0/0 (3.0)
Epoch 003/061 | Loss 1.7437 | Win/lose count 3.5/3.0 (0.5)
Epoch 004/061 | Loss 1.7258 | Win/lose count 8.5/4.0 (4.5)
Epoch 005/061 | Loss 1.7896 | Win/lose count 4.5/4.0 (0.5)
Epoch 006/061 | Loss 1.7889 | Win/lose count 4.5/2.0 (2.5)
Epoch 007/061 | Loss 1.9408 | Win/lose count 4.5/3.0 (1.5)
Epoch 008/061 | Loss 1.8073 | Win/lose count 4.5/1.0 (3.5)
Epoch 009/061 | Loss 1.7679 | Win/lose count 5.0/0 (5.0)
Epoch 010/061 | Loss 1.7656 | Win/lose count 4.5/1.0 (3.5)
Epoch 011/061 | Loss 1.8645 | Win/lose count 2.5/4.0 (-1.5)
Epoch 012/061 | Loss 1.7341 | Win/lose count 0.5/2.0 (-1.5)
Epoch 013/061 | Loss 1.6809 | Win/lose count 4.0/2.0 (2.0)
Epoch 014/061 | Loss 1.7823 | Win/lose count 2.5/4.0 (-1.5)
Epoch 015/061 | Loss 1.7332 | Win/lose count 5.5/4.0 (1.5)
Epoch 016/061 | Loss 1.7814 | Win/lose count 8.0/6.0 (2.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
from keras.layers import Flatten

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(8, (5,5),input_shape = (5, 5, self.n_state), padding = 'same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(1,1)))
        model.add(Conv2D(16, (5,5), padding = 'same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(1,1)))
        model.add(Conv2D(32, (5,5), padding = 'same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(1,1)))
        model.add(Flatten())
        model.add(Dense(24, activation = "relu"))
        model.add(Dense(4, activation = "linear"))        
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(Adam(lr=lr), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, n_state = 2, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train50.mp4'))

Epoch 000/061 | Loss 1.7499 | Win/lose count 1.5/2.0 (-0.5)
Epoch 001/061 | Loss 1.2883 | Win/lose count 5.5/7.0 (-1.5)
Epoch 002/061 | Loss 1.5102 | Win/lose count 5.0/10.0 (-5.0)
Epoch 003/061 | Loss 1.5449 | Win/lose count 5.0/6.0 (-1.0)
Epoch 004/061 | Loss 1.8203 | Win/lose count 5.5/7.0 (-1.5)
Epoch 005/061 | Loss 1.7217 | Win/lose count 10.0/6.0 (4.0)
Epoch 006/061 | Loss 1.7667 | Win/lose count 5.5/2.0 (3.5)
Epoch 007/061 | Loss 1.7228 | Win/lose count 7.5/3.0 (4.5)
Epoch 008/061 | Loss 1.8424 | Win/lose count 6.5/3.0 (3.5)
Epoch 009/061 | Loss 1.8183 | Win/lose count 4.0/2.0 (2.0)
Epoch 010/061 | Loss 1.4641 | Win/lose count 1.5/0 (1.5)
Epoch 011/061 | Loss 1.8874 | Win/lose count 9.0/2.0 (7.0)
Epoch 012/061 | Loss 1.9566 | Win/lose count 9.5/4.0 (5.5)
Epoch 013/061 | Loss 1.8679 | Win/lose count 9.0/6.0 (3.0)
Epoch 014/061 | Loss 2.0008 | Win/lose count 7.0/1.0 (6.0)
Epoch 015/061 | Loss 1.9253 | Win/lose count 6.0/2.0 (4.0)
Epoch 016/061 | Loss 2.0614 | Win/lose count 12.5/2

In [16]:
test(agent,env,epochs_test,prefix='cnn_test')
HTML(display_videos('cnn_test10.mp4'))

Win/lose count 7.5/0. Average score (7.5)
Win/lose count 4.5/0. Average score (6.0)
Win/lose count 5.0/1.0. Average score (5.333333333333333)
Win/lose count 7.5/0. Average score (5.875)
Win/lose count 6.5/0. Average score (6.0)
Win/lose count 4.0/0. Average score (5.666666666666667)
Win/lose count 1.0/0. Average score (5.0)
Win/lose count 2.0/0. Average score (4.625)
Win/lose count 11.5/2.0. Average score (5.166666666666667)
Win/lose count 1.0/0. Average score (4.75)
Win/lose count 15.5/0. Average score (5.7272727272727275)
Final score: 5.7272727272727275


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.001, n_state=2, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.001, n_state=2, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 22.0/1.0. Average score (21.0)
Win/lose count 7.0/1.0. Average score (13.5)
Win/lose count 13.5/0. Average score (13.5)
Win/lose count 5.0/0. Average score (11.375)
Win/lose count 15.5/0. Average score (12.2)
Win/lose count 22.0/0. Average score (13.833333333333334)
Win/lose count 2.0/1.0. Average score (12.0)
Win/lose count 25.5/2.0. Average score (13.4375)
Win/lose count 4.5/0. Average score (12.444444444444445)
Win/lose count 9.5/1.0. Average score (12.05)
Win/lose count 9.0/0. Average score (11.772727272727273)
Final score: 11.772727272727273
Test of the FC
Win/lose count 20.0/0. Average score (20.0)
Win/lose count 0.5/1.0. Average score (9.75)
Win/lose count 18.5/0. Average score (12.666666666666666)
Win/lose count 16.0/0. Average score (13.5)
Win/lose count 8.0/0. Average score (12.4)
Win/lose count 19.0/0. Average score (13.5)
Win/lose count 18.0/0. Average score (14.142857142857142)
Win/lose count 27.0/1.0. Average score (15.625)
Win/lose count 12

In [18]:
HTML(display_videos('cnn_test10.mp4'))

In [19]:
HTML(display_videos('fc_test10.mp4'))

>The fully connected layers agent seems to reach a better performance on testing than the convolutionnal agent.
>The main issue occurs when the mouse cannot see any red square next to it. When this event occurs, the mouse is stuck for a long time.
>The higher the temperature, the better the fc algorithm compared to the cnn. On the contrary, the lower the temperature, the better CNN is compared to fc.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [23]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        #Update of the agent.Learning_rate
        if (e%10 == 0):
            agent.Learning_rate = 1/(e/10+1)
            print(agent.Learning_rate)
        #Update of the agent.epsilon
        if (e%10 == 0):
            agent.set_epsilon(2*agent.epsilon/(e/10+1))
            print(agent.epsilon)
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            
            # add train = True
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        # add malus_position 
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        # Change reward calculation
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.3

        reward = reward + self.board[self.x, self.y]
        
        
        self.t = self.t + 1
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        # add malus_position 
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [24]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1,n_state=3 , epsilon = 0.1, memory_size=2000, batch_size = 32)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

1.0
0.2
Epoch 000/061 | Loss 0.0323 | Win/lose count 11.5/54.5999999999999 (-43.0999999999999)
Epoch 001/061 | Loss 1.8557 | Win/lose count 11.0/54.89999999999988 (-43.89999999999988)
Epoch 002/061 | Loss 1.8819 | Win/lose count 7.5/53.19999999999988 (-45.69999999999988)
Epoch 003/061 | Loss 1.7418 | Win/lose count 14.5/53.39999999999986 (-38.89999999999986)
Epoch 004/061 | Loss 1.8018 | Win/lose count 13.0/55.099999999999845 (-42.099999999999845)
Epoch 005/061 | Loss 0.8921 | Win/lose count 7.0/59.39999999999981 (-52.39999999999981)
Epoch 006/061 | Loss 1.7447 | Win/lose count 12.5/48.89999999999994 (-36.39999999999994)
Epoch 007/061 | Loss 1.9038 | Win/lose count 12.5/50.79999999999992 (-38.29999999999992)
Epoch 008/061 | Loss 1.7713 | Win/lose count 16.0/50.59999999999989 (-34.59999999999989)
Epoch 009/061 | Loss 1.7801 | Win/lose count 12.0/54.89999999999986 (-42.89999999999986)
0.5
0.2
Epoch 010/061 | Loss 1.8754 | Win/lose count 8.0/56.799999999999855 (-48.799999999999855)
Epoch 

In [25]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 0/61.79999999999977. Average score (-61.79999999999977)
Win/lose count 2.5/58.299999999999805. Average score (-58.799999999999784)
Win/lose count 0/59.69999999999979. Average score (-59.09999999999979)
Win/lose count 0/61.199999999999775. Average score (-59.62499999999979)
Win/lose count 1.0/59.39999999999979. Average score (-59.37999999999979)
Win/lose count 0.5/60.299999999999784. Average score (-59.44999999999979)
Win/lose count 1.0/59.59999999999979. Average score (-59.328571428571216)
Win/lose count 1.5/59.899999999999785. Average score (-59.21249999999979)
Win/lose count 0/59.69999999999979. Average score (-59.26666666666646)
Win/lose count 1.0/59.899999999999785. Average score (-59.22999999999979)
Win/lose count 2.5/58.8999999999998. Average score (-58.97272727272706)
Final score: -58.97272727272706


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***